# Improve your Score with some Text Preprocessing


This kernel is an improved version of @Dieter's work.
> https://www.kaggle.com/christofhenkel/how-to-preprocessing-when-using-embeddings

This is also how I caught a glimpse of spelling mistakes in the database.

In [16]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

import operator 
import re

## Loading data

In [ ]:
df = pd.read_csv('../data/data.tsv', sep='\t', header=None, names=['query_id', 'query_text', 'passage_text', 'label', 'passage_id'])
df = df[[ 'query_text', 'passage_text']]
print(df.shape)
df.head()

In [25]:
df = pd.DataFrame(np.reshape(df.values, (-1,1)), columns=['text'])
print(df.shape)
df.head()

(10483760, 1)


,text
0,. what is a corporation?
1,A company is incorporated in a specific nation...
2,. what is a corporation?
3,"Today, there is a growing community of more th..."
4,. what is a corporation?


## Loading embeddings

In [ ]:
# http://nlp.stanford.edu/data/glove.840B.300d.zip
# paragram
# https://drive.google.com/file/d/0B9w48e1rj-MOck1fRGxaZW1LU2M/view?usp=sharing
# https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki-news-300d-1M.vec.zip
# https://s3-us-west-1.amazonaws.com/fasttext-vectors/crawl-300d-2M.vec.zip

In [10]:
def load_embed(file):
    def get_coefs(word,*arr): 
        return word, np.asarray(arr, dtype='float32')
    
    if file == '../embeddings/crawl-300d-2M.vec':
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file, encoding='utf-8') if len(o)>100)
    else:
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file, encoding='latin'))
        
    return embeddings_index

In [11]:
glove = '../embeddings/glove.840B.300d.txt'
paragram =  '../embeddings/paragram_300_sl999.txt'
wiki_news = '../embeddings/crawl-300d-2M.vec'

In [ ]:
print("Extracting GloVe embedding")
embed_glove = load_embed(glove)
print("Extracting Paragram embedding")
embed_paragram = load_embed(paragram)

In [12]:
print("Extracting FastText embedding")
embed_fasttext = load_embed(wiki_news)

Extracting FastText embedding


In [19]:
cosine(embed_glove['the'], embed_glove['the'])

0.0

In [20]:
cosine(embed_paragram['the'], embed_glove['the'])

0.565854549407959

In [21]:
cosine(embed_paragram['like'], embed_glove['like'])

0.3260217308998108

## Vocabulary and Coverage functions

In [26]:
def build_vocab(texts):
    sentences = texts.apply(lambda x: x.split()).values
    vocab = {}
    for sentence in sentences:
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [27]:
def check_coverage(vocab, embeddings_index):
    known_words = {}
    unknown_words = {}
    nb_known_words = 0
    nb_unknown_words = 0
    for word in vocab.keys():
        try:
            known_words[word] = embeddings_index[word]
            nb_known_words += vocab[word]
        except:
            unknown_words[word] = vocab[word]
            nb_unknown_words += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(known_words) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(nb_known_words / (nb_known_words + nb_unknown_words)))
    unknown_words = sorted(unknown_words.items(), key=operator.itemgetter(1))[::-1]

    return unknown_words

## Starting point

In [28]:
vocab = build_vocab(df['text'])

In [29]:
print("Glove : ")
oov_glove = check_coverage(vocab, embed_glove)
print("Paragram : ")
oov_paragram = check_coverage(vocab, embed_paragram)
print("FastText : ")
oov_fasttext = check_coverage(vocab, embed_fasttext)

Glove : 
Found embeddings for 14.40% of vocab
Found embeddings for  88.75% of all text
Paragram : 
Found embeddings for 7.60% of vocab
Found embeddings for  77.39% of all text
FastText : 
Found embeddings for 15.16% of vocab
Found embeddings for  90.50% of all text


 #### Paragram seems to have a significantly lower coverage. 
>That's because it does not understand upper letters, let us lower our texts :

In [30]:
df['lowered_text'] = df['text'].apply(lambda x: x.lower())

In [31]:
vocab_low = build_vocab(df['lowered_text'])

In [32]:
print("Glove : ")
oov_glove = check_coverage(vocab_low, embed_glove)
print("Paragram : ")
oov_paragram = check_coverage(vocab_low, embed_paragram)
print("FastText : ")
oov_fasttext = check_coverage(vocab_low, embed_fasttext)

Glove : 
Found embeddings for 9.16% of vocab
Found embeddings for  88.62% of all text
Paragram : 
Found embeddings for 12.58% of vocab
Found embeddings for  89.07% of all text
FastText : 
Found embeddings for 9.40% of vocab
Found embeddings for  90.44% of all text


#### Better, but we lost a bit of information on the other embeddings.
> Therer are words known that are known with upper letters and unknown without. Let us fix that :
- word.lower() takes the embedding of word if word.lower() doesn't have an embedding

In [33]:
def add_lower(embedding, vocab):
    count = 0
    for word in vocab:
        if word in embedding and word.lower() not in embedding:  
            embedding[word.lower()] = embedding[word]
            count += 1
    print(f"Added {count} words to embedding")

In [34]:
print("Glove : ")
add_lower(embed_glove, vocab)
print("Paragram : ")
add_lower(embed_paragram, vocab)
print("FastText : ")
add_lower(embed_fasttext, vocab)

Glove : 
Added 134402 words to embedding
Paragram : 
Added 25 words to embedding
FastText : 
Added 165456 words to embedding


In [35]:
print("Glove : ")
oov_glove = check_coverage(vocab_low, embed_glove)
print("Paragram : ")
oov_paragram = check_coverage(vocab_low, embed_paragram)
print("FastText : ")
oov_fasttext = check_coverage(vocab_low, embed_fasttext)

Glove : 
Found embeddings for 12.30% of vocab
Found embeddings for  89.06% of all text
Paragram : 
Found embeddings for 12.58% of vocab
Found embeddings for  89.07% of all text
FastText : 
Found embeddings for 13.27% of vocab
Found embeddings for  90.86% of all text


### What's wrong ?

In [37]:
oov_glove[:50]

[('–', 163115),
 ('however,', 127764),
 ('—', 91547),
 ('example,', 80868),
 ('…', 75125),
 ('it’s', 69085),
 ('county,', 57218),
 ('(or', 54313),
 ('(the', 48567),
 ('time,', 47839),
 ('•', 43404),
 ('year,', 42919),
 ('years,', 37443),
 ('include:', 35574),
 ('states,', 35568),
 ('answer:', 35090),
 ("what's", 34995),
 ('don’t', 34274),
 ('you’re', 33342),
 ('day,', 32410),
 ('1,', 31477),
 ('water,', 30212),
 ('is,', 29392),
 ('system,', 27805),
 ("world's", 26538),
 ('city,', 26202),
 ('(a', 26195),
 ('and,', 25762),
 ('·', 25609),
 ('is:', 24803),
 ('disease,', 24681),
 ('pain,', 24650),
 ('(also', 23917),
 ('body,', 23820),
 ('cases,', 23743),
 ('(in', 23606),
 ('fact,', 22711),
 ('days,', 22257),
 ('(see', 22239),
 ("earth's", 21871),
 ('(and', 21834),
 ('so,', 21758),
 ('definition:', 21595),
 ('that’s', 21225),
 ('you’ll', 20966),
 ('symptoms.', 20945),
 ('2015.', 20621),
 ('name,', 20355),
 ('cells,', 20165),
 ('age,', 19485)]

#### First faults appearing are : 
- Contractions 
- Words with punctuation in them

> Let us correct that.

## Contractions

In [38]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }

In [39]:
def known_contractions(embed):
    known = []
    for contract in contraction_mapping:
        if contract in embed:
            known.append(contract)
    return known

In [40]:
print("- Known Contractions -")
print("   Glove :")
print(known_contractions(embed_glove))
print("   Paragram :")
print(known_contractions(embed_paragram))
print("   FastText :")
print(known_contractions(embed_fasttext))

- Known Contractions -
   Glove :
["can't", "'cause", "didn't", "doesn't", "don't", "I'd", "I'll", "I'm", "I've", "i'd", "i'll", "i'm", "i've", "it's", "ma'am", "o'clock", "that's", "you'll", "you're"]
   Paragram :
["can't", "'cause", "didn't", "doesn't", "don't", "i'd", "i'll", "i'm", "i've", "it's", "ma'am", "o'clock", "that's", "you'll", "you're"]
   FastText :
["can't", "'cause", "didn't", "doesn't", "don't", "haven't", "he's", "I'd", "I'll", "I'm", "I've", "i'd", "i'll", "i'm", "i've", "isn't", "it's", "she's", "that's", "there's", "they're", "we're", "won't", "you'll", "you're"]


#### FastText does not understand contractions
> We use the map to replace them

In [41]:
def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

In [42]:
df['treated_text'] = df['lowered_text'].apply(lambda x: clean_contractions(x, contraction_mapping))

In [44]:
vocab = build_vocab(df['treated_text'])
print("Glove : ")
oov_glove = check_coverage(vocab, embed_glove)
print("Paragram : ")
oov_paragram = check_coverage(vocab, embed_paragram)
print("FastText : ")
oov_fasttext = check_coverage(vocab, embed_fasttext)

Glove : 
Found embeddings for 12.41% of vocab
Found embeddings for  89.27% of all text
Paragram : 
Found embeddings for 12.68% of vocab
Found embeddings for  89.28% of all text
FastText : 
Found embeddings for 13.38% of vocab
Found embeddings for  90.98% of all text


## Now, let us deal with special characters

In [45]:
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'

In [46]:
def unknown_punct(embed, punct):
    unknown = ''
    for p in punct:
        if p not in embed:
            unknown += p
            unknown += ' '
    return unknown

In [47]:
print("Glove :")
print(unknown_punct(embed_glove, punct))
print("Paragram :")
print(unknown_punct(embed_paragram, punct))
print("FastText :")
print(unknown_punct(embed_fasttext, punct))

Glove :
“ ” ’ ∞ θ ÷ α • à − β ∅ ³ π ‘ ₹ ´ ° £ € × ™ √ ² — – 
Paragram :
“ ” ’ ∞ θ ÷ α • à − β ∅ ³ π ‘ ₹ ´ ° £ € × ™ √ ² — – 
FastText :
_ ` 


#### FastText seems to have a better knowledge of special characters 
> We use a map to replace unknown characters with known ones.

> We make sure there are spaces between words and punctuation


In [48]:
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

In [49]:
def clean_special_chars(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}  # Other special characters that I have to deal with in last
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text

In [51]:
df['treated_text'] = df['treated_text'].apply(lambda x: clean_special_chars(x, punct, punct_mapping))

In [53]:
vocab = build_vocab(df['treated_text'])
print("Glove : ")
oov_glove = check_coverage(vocab, embed_glove)
print("Paragram : ")
oov_paragram = check_coverage(vocab, embed_paragram)
print("FastText : ")
oov_fasttext = check_coverage(vocab, embed_fasttext)

Glove : 
Found embeddings for 37.70% of vocab
Found embeddings for  99.51% of all text
Paragram : 
Found embeddings for 42.79% of vocab
Found embeddings for  99.54% of all text
FastText : 
Found embeddings for 38.64% of vocab
Found embeddings for  99.58% of all text


In [54]:
oov_fasttext[:100]

[('47474', 3242),
 ('sylls', 3027),
 ('efinition', 2865),
 ('com®', 2765),
 ('allacronyms', 2507),
 ('syll', 2208),
 ('pagessm', 2111),
 ('audioenglish', 1562),
 ('onfidence', 1148),
 ('soulurge', 961),
 ('irections', 877),
 ('powerthesaurus', 838),
 ('itamin', 752),
 ('epending', 700),
 ('onlineslangdictionary', 679),
 ('1½', 662),
 ('auses', 649),
 ('59½', 636),
 ('wunderphotos', 630),
 ('wutv', 625),
 ('noncount', 625),
 ('homewyse', 622),
 ('eaning', 612),
 ('rewardsfor', 585),
 ('››', 547),
 ('nstructions', 537),
 ('ypically', 533),
 ('motion®', 504),
 ('lexilogos', 503),
 ('mayoclinic', 488),
 ('censusviewer', 483),
 ('2″', 441),
 ('ummary', 440),
 ('mail®', 440),
 ('acronymfinder', 418),
 ('4″', 408),
 ('70½', 400),
 ('wikianswers®', 399),
 ('uszip', 397),
 ('1⁄2', 395),
 ('windows®', 390),
 ('©1988', 385),
 ('5′', 385),
 ('iconsimon', 378),
 ('yourselfers', 378),
 ('intel®', 366),
 ('lexibase', 365),
 ('ackground', 364),
 ('zpass', 363),
 ('omments', 363),
 ('uscle', 355),
 ('i

### What's still missing ? 
- Unknown words
- Acronyms
- Spelling mistakes

## We can correct manually most frequent mispells

#### For example, here are some mistakes and their frequency
- qoura : 85 times
- mastrubation : 38 times
- demonitisation : 30 times
- …

In [55]:
mispell_dict1 = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}

In [ ]:
mispell_dict2 = {''}
mispell_dict = {**mispell_dict1, **mispell_dict2}

In [56]:
def correct_spelling(x, dic):
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x

In [58]:
df['treated_text'] = df['treated_text'].apply(lambda x: correct_spelling(x, mispell_dict))

In [60]:
vocab = build_vocab(df['treated_text'])
print("Glove : ")
oov_glove = check_coverage(vocab, embed_glove)
print("Paragram : ")
oov_paragram = check_coverage(vocab, embed_paragram)
print("FastText : ")
oov_fasttext = check_coverage(vocab, embed_fasttext)

Glove : 
Found embeddings for 37.70% of vocab
Found embeddings for  99.51% of all text
Paragram : 
Found embeddings for 42.78% of vocab
Found embeddings for  99.54% of all text
FastText : 
Found embeddings for 38.63% of vocab
Found embeddings for  99.58% of all text


### That's all for now !

#### Improvement ideas: 
> Replace acronyms with their meaning

> Replace unknown words with a more general term : 
 - ex : fortnite, pubg -> video game
 
 ### *Thanks for reading ! *